<a href="https://colab.research.google.com/github/markaaronslater/recurrent-NMT/blob/master/NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from encoderdecoder import *
from batch import *
from traintest import *
from processCorpuses import *

# from batch import *
# from processCorpuses import *
# from unrolled_encdec import *
# from unrolled_traintest import *


%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!nvidia-smi

Thu Aug 27 22:02:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# inserted a cell!!!

In [ ]:
pip install subword-nmt

In [56]:
path = '/content/gdrive/My Drive/iwslt16_en_de/' # path to corpuses
#path = '/content/gdrive/My Drive/toy_corpuses/' # path to toy corpuses

norm_texts = normalizeCorpuses(path)

train_src_sentences, train_trg_sentences, dev_src_sentences, dev_trg_sentences, test_src_sentences = norm_texts[0], norm_texts[1], norm_texts[2], norm_texts[3], norm_texts[4]
#train_src_sentences, train_trg_sentences = norm_texts[0], norm_texts[1]

references = load_docs(path, tok=True)

#print("train_references:")
#print(references[1])

train_references = []
for ref in references[1]:
    # # hacky workaround: dn include newline that tokenizeCorpuses added to tok_train
    # for ref in references[1][:-1]:
    #print(ref)
    ref = ref.replace(" - ", "-")
    train_references.append([ref.split()])
    #print(train_references)
    
dev_references = []
for ref in references[3]:
    ref = ref.replace(" - ", "-")
    dev_references.append([ref.split()])
    
print()
for sent in train_trg_sentences[:5]:
    print(sent)
for sent in train_references[:5]:
    print(sent)    
print()
for sent in dev_trg_sentences[:5]:
    print(sent)    
for sent in dev_references[:5]:
    print(sent)   
print()

train_src_sentences = [sent.split() for sent in train_src_sentences]
train_trg_sentences = [sent.split() for sent in train_trg_sentences]
dev_src_sentences = [sent.split() for sent in dev_src_sentences]
dev_trg_sentences = [sent.split() for sent in dev_trg_sentences]
test_src_sentences = [sent.split() for sent in test_src_sentences]

196884 pairs before filtering
180 songs, 115 overly long
196589 pairs after filtering
tokenizing train.de...
tokenizing train.en...
tokenizing dev.de...
tokenizing dev.en...
tokenizing test.de...
creating de names table...
creating en names table...
decasing train.de...
decasing train.en...
decasing dev.de...
decasing dev.en...
decasing test.de...

David Gallo : this is Bill Lange . I'm Dave Gallo .
and we're going to tell you some stories from the sea here in video .
we've got some of the most incredible video of Titanic that's ever been seen , and we're not going to show you any of it .
the truth of the matter is that the Titanic -- even though it's breaking all sorts of box office records -- it's not the most exciting story from the sea .
and the problem , I think , is that we take the ocean for granted .
[['David', 'Gallo', ':', 'This', 'is', 'Bill', 'Lange', '.', "I'm", 'Dave', 'Gallo', '.']]
[['And', "we're", 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 

In [8]:
print(train_src_sentences)

[['Etre'], ['X', 'y', 'z'], ['Ce', 'soir']]


In [9]:
print(train_trg_sentences)

[['To', 'be', 'free'], ['C', 'd'], ['E']]


In [57]:
for sent in train_trg_sentences[:5]:
    print(sent)
for sent in train_references[:5]:
    print(sent)    
print()
for sent in dev_trg_sentences[:5]:
    print(sent)    
for sent in dev_references[:5]:
    print(sent)   
print()

['David', 'Gallo', ':', 'this', 'is', 'Bill', 'Lange', '.', "I'm", 'Dave', 'Gallo', '.']
['and', "we're", 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 'here', 'in', 'video', '.']
["we've", 'got', 'some', 'of', 'the', 'most', 'incredible', 'video', 'of', 'Titanic', "that's", 'ever', 'been', 'seen', ',', 'and', "we're", 'not', 'going', 'to', 'show', 'you', 'any', 'of', 'it', '.']
['the', 'truth', 'of', 'the', 'matter', 'is', 'that', 'the', 'Titanic', '--', 'even', 'though', "it's", 'breaking', 'all', 'sorts', 'of', 'box', 'office', 'records', '--', "it's", 'not', 'the', 'most', 'exciting', 'story', 'from', 'the', 'sea', '.']
['and', 'the', 'problem', ',', 'I', 'think', ',', 'is', 'that', 'we', 'take', 'the', 'ocean', 'for', 'granted', '.']
[['David', 'Gallo', ':', 'This', 'is', 'Bill', 'Lange', '.', "I'm", 'Dave', 'Gallo', '.']]
[['And', "we're", 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 'here', 'in', 'video', '.']]
[["We've", 'got',

In [58]:


#embType = 'jointBPE'
embType = 'word'

if embType == 'word':
  
    

    src_counter, trg_counter = to_counters(train_src_sentences, train_trg_sentences)
    #print(trg_counter)
    trim_type = "topK"
    #trim_type = "threshold"
    if trim_type == "threshold":
        st = 5
        tt = 5
        # st = 1 # for debug
        # tt = 1
        trimmed_src_vocab, trimmed_trg_vocab = trim_vocabs(src_counter, trg_counter, srcThres=st, trgThres=tt)
    elif trim_type == "topK":
        trimmed_src_vocab, trimmed_trg_vocab = trim_vocabs2(src_counter, trg_counter, srcK=60000, trgK=45000)

    train_src_sentences = removeOOV(train_src_sentences, trimmed_src_vocab)
    train_trg_sentences = removeOOV(train_trg_sentences, trimmed_trg_vocab)
    dev_src_sentences = removeOOV(dev_src_sentences, trimmed_src_vocab)
    test_src_sentences = removeOOV(test_src_sentences, trimmed_src_vocab)

    train_trg_sentences = add_start_end_tokens(train_trg_sentences)
    srcV, trgV, idx_to_src_word, idx_to_trg_word = computeVocabs(train_src_sentences, train_trg_sentences)

    train_src_sentences = toIndices(train_src_sentences, srcV)
    train_trg_sentences = toIndices(train_trg_sentences, trgV)
    dev_src_sentences = toIndices(dev_src_sentences, srcV)   
    test_src_sentences = toIndices(test_src_sentences, srcV)





lengths of vocabs before trimming: src: 120990, trg: 56561
lengths of vocabs after trimming: src: 60000, trg: 45000


In [59]:
print(train_src_sentences[:10])
print()
print(train_trg_sentences[:10])

[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 3, 9], [13, 14, 15, 16, 17, 18, 5, 19, 20, 21, 22, 9], [13, 23, 24, 25, 26, 27, 28, 26, 29, 30, 31, 32, 33, 34, 35, 30, 30, 36, 13, 14, 15, 37, 38, 39, 9], [31, 40, 6, 30, 41, 31, 29, 42, 43, 44, 45, 46, 47, 48, 31, 49, 50, 51, 19, 6, 9], [10, 52, 30, 5, 53, 6, 30, 41, 13, 5, 19, 54, 55, 56, 57, 9], [58, 32, 59, 60, 30, 61, 31, 62, 63, 64, 65, 66, 9], [26, 67, 26, 68, 6, 69, 9], [31, 70, 71, 6, 72, 73, 74, 9], [24, 75, 64, 76, 6, 30, 41, 13, 77, 78, 79, 80, 81, 82, 83, 84, 85, 36, 86, 31, 87, 88, 89, 90, 30, 36, 43, 91, 36, 92, 93, 30, 94, 95, 96, 30, 97, 36, 98, 30, 99, 13, 23, 100, 101, 30, 102, 103, 104, 6, 9], [20, 105, 106, 107, 93, 31, 108, 109, 64, 65, 9]]

[[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 5, 11, 3], [2, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 11, 3], [2, 28, 29, 20, 30, 23, 31, 32, 27, 30, 33, 34, 35, 36, 37, 38, 14, 15, 39, 16, 17, 40, 19, 41, 30, 42, 11, 3], [2, 23, 43, 30, 23, 44, 8, 45, 23, 33, 46, 47, 48, 49, 50, 

In [ ]:
for i, word in enumerate(srcV):
  if i == 120:
    break
  print(word, ": ", srcV[word])

In [ ]:
for i, word in enumerate(trgV):
  if i == 120:
    break
  print(word, ": ", trgV[word])

In [ ]:
if embType == 'jointBPE':
    alreadyExists = False
    if not alreadyExists:
        numMerges = 30000
        vocabThreshold = 10
        !bash jointBPE.sh 30000 10 '/content/gdrive/My Drive/iwslt16_en_de/'



    
    

learning joint bpe and vocab using 30000 merge operations...
applying bpe with vocab threshold of 10 to train...
applying bpe with vocab threshold of 10 to dev and test...
done


In [ ]:
path = '/content/gdrive/My Drive/iwslt16_en_de/' # path to corpuses

bpe_texts = load_docs(path, bpe=True)
train_src_sentences, train_trg_sentences, dev_src_sentences, dev_trg_sentences, test_src_sentences = bpe_texts[0], bpe_texts[1], bpe_texts[2], bpe_texts[3], bpe_texts[4]

train_references = []
for ref in train_trg_sentences[:]: # until think of better way
    ref = ref.replace(" - ", "-")
    train_references.append([ref.split()])
    
dev_references = []
for ref in dev_trg_sentences[:]:
    ref = ref.replace(" - ", "-")
    dev_references.append([ref.split()])
    

for sent in train_trg_sentences[:5]:
    print(sent)
for sent in train_references[:5]:
    print(sent)    
print()
for sent in dev_trg_sentences[:5]:
    print(sent)    
for sent in dev_references[:5]:
    print(sent)   
print()

train_src_sentences = [sent.split() for sent in train_src_sentences]
train_trg_sentences = [sent.split() for sent in train_trg_sentences]
dev_src_sentences = [sent.split() for sent in dev_src_sentences]
dev_trg_sentences = [sent.split() for sent in dev_trg_sentences]
test_src_sentences = [sent.split() for sent in test_src_sentences]


train_trg_sentences = add_start_end_tokens(train_trg_sentences)

vocab, idx_to_subword = computeBPEvocabs(path + "vocab.de", path + "vocab.en")

train_src_sentences = toIndices(train_src_sentences, vocab)
train_trg_sentences = toIndices(train_trg_sentences, vocab)
dev_src_sentences = toIndices(dev_src_sentences, vocab)   
test_src_sentences = toIndices(test_src_sentences, vocab)



David Gall@@ o : this is Bill L@@ ange . I'm Dave Gall@@ o .
and we're going to tell you some stories from the sea here in video .
we've got some of the most incredible video of Titanic that's ever been seen , and we're not going to show you any of it .
the truth of the matter is that the Titanic -- even though it's breaking all sorts of box office records -- it's not the most exciting story from the sea .
and the problem , I think , is that we take the ocean for granted .
[['David', 'Gall@@', 'o', ':', 'this', 'is', 'Bill', 'L@@', 'ange', '.', "I'm", 'Dave', 'Gall@@', 'o', '.']]
[['and', "we're", 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 'here', 'in', 'video', '.']]
[["we've", 'got', 'some', 'of', 'the', 'most', 'incredible', 'video', 'of', 'Titanic', "that's", 'ever', 'been', 'seen', ',', 'and', "we're", 'not', 'going', 'to', 'show', 'you', 'any', 'of', 'it', '.']]
[['the', 'truth', 'of', 'the', 'matter', 'is', 'that', 'the', 'Titanic', '--', 'even', 'tho

In [69]:
# build model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

overfit_on_train = True

if overfit_on_train:
    b = 10
    #dev_bsz = 10
    dev_bsz = 1
else:
    b = 64
    dev_bsz = 64
    #dev_bsz = 1
num_ep = 300 # number of epochs
IS = 300 # input size
enc_hs = 600 # encoder hidden size
dec_hs = 600 # decoder hidden size
enc_NL = 1 # number of layers
dec_NL = 1
enc_DR = .0 # dropout
dec_DR = .0
bi_enc = True
project = True
reverse_src = False
#tie_weights = True
tie_weights = False


customLSTM = False
#embType = "jointBPE"
embType = "word"
trim_type = "threshold"
inputFeeding = True
K = .7
LR = .01 # learning rate
WD = .0

enc_emb_drop = .1
dec_emb_drop = .1
dropconnect = .5

opt = "ADAM" # optimization algorithm
model_name = 'debug_model/'
msg = ' \n'


encoder_params = {}
if embType == "word":
    encoder_params['vocab_size'] = len(srcV)
elif embType == "jointBPE":
    encoder_params['vocab_size'] = len(vocab)
encoder_params['input_size'] = IS
encoder_params['hidden_size'] = enc_hs
encoder_params['num_layers'] = enc_NL
encoder_params['dropout'] = enc_DR
encoder_params['dev'] = device
encoder_params['bi_enc'] = bi_enc
encoder_params['project'] = project # project concated enc states to dim of decoder
#encoder_params['src_emb_drop'] = enc_emb_drop
#encoder_params['dropconnect'] = dropconnect
encoder_params['reverse_src'] = reverse_src
#encoder_params['variational_drop'] = .5
#encoder_params['i0_drop'] = .2
#encoder_params['i1_drop'] = .5
encoder_params['out_drop'] = .5
encoder_params['init_scheme'] = 'layer_to_layer'
#encoder_params['weights_to_drop'] = ['weight_hh_l0']
encoder_params['customLSTM'] = customLSTM




decoder_params = {}
if embType == "word":
    decoder_params['sosIdx'] = trgV['<sos>']
    #print(decoder_params['sosIdx'])
    decoder_params['eosIdx'] = trgV['<eos>']
    #print(decoder_params['eosIdx'])

    decoder_params['padIdx'] = trgV['<pad>']
    #print(decoder_params['padIdx'])

    decoder_params['vocab_size'] = len(trgV)
    decoder_params['idx_to_trg_word'] = idx_to_trg_word

elif embType == "jointBPE":

    decoder_params['sosIdx'] = vocab['<sos>']
    #print(decoder_params['sosIdx'])
    decoder_params['eosIdx'] = vocab['<eos>']
    #print(decoder_params['eosIdx'])

    decoder_params['padIdx'] = vocab['<pad>']
    #print(decoder_params['padIdx'])

    decoder_params['vocab_size'] = len(vocab)
    decoder_params['idx_to_trg_word'] = idx_to_subword




decoder_params['input_size'] = IS
decoder_params['hidden_size'] = dec_hs
decoder_params['num_layers'] = dec_NL
decoder_params['dropout'] = dec_DR
#decoder_params['trg_emb_drop'] = dec_emb_drop
#decoder_params['dropconnect'] = dropconnect

decoder_params['dev'] = device
#decoder_params['attention'] = "global_att"
decoder_params['attention'] = None


decoder_params['inf_alg'] = "greedy_search"
#decoder_params['inf_alg'] = "beam_search"

decoder_params['beam_size'] = 10
#decoder_params['decode_slack'] = 8
decoder_params['decode_slack'] = 20

#decoder_params['variational_drop'] = .5
#decoder_params['i0_drop'] = .0 # applied to trg embeddings prior to entering decoder lstm
#decoder_params['i1_drop'] = .5
decoder_params['out_drop'] = .5 # applied after final layer of decoder lstm
#decoder_params['att_drop'] = .3 # applied after project back to input size
#decoder_params['rec_drop'] = .5

decoder_params['tie_weights'] = tie_weights
#decoder_params['weights_to_drop'] = ['weight_hh_l0']
decoder_params['customLSTM'] = customLSTM
#decoder_params['input_feeding'] = inputFeeding
#decoder_params['K'] = K




encoder = Encoder(encoder_params)
decoder = Decoder(decoder_params)
translator = RNNencdec(encoder, decoder, embType) # initialize model

if torch.cuda.is_available():
    translator.cuda()
    

optimizer = torch.optim.Adam(translator.parameters(), lr=LR, weight_decay=WD) # initialize optimizer

# folder to write checkpoints (train outputs) and translations (inference outputs)
folder = '/content/gdrive/My Drive/MToutputs/' + model_name

cuda:0


In [70]:

import os
if not os.path.exists(folder):
    os.makedirs(folder)

x = !nvidia-smi | head -n 8 | tail -n 1 | cut -d ' ' -f 6-7
current_gpu = ' '.join(x)
with open(folder + 'model_train_stats.txt', 'w') as f:
    f.write(msg)
    if trim_type == "threshold":
        f.write("srcThres: {}\n".format(st))
        f.write("trgThres: {}\n".format(tt))

    f.write("encoder_params:\n")
    for key in encoder_params:
        f.write(key + ": {}\n".format(encoder_params[key]))
    f.write("\n")
    f.write("decoder_params:\n")
    for key in decoder_params:
        if key != "idx_to_trg_word":
            f.write(key + ": {}\n".format(decoder_params[key]))
        
    f.write("\n")
    f.write("embType: {}\n".format(embType))
    f.write("lr: {}\n".format(LR))
    f.write("wd: {}\n".format(WD))
    f.write("opt: {}\n".format(opt))
    f.write("\n")
    f.write("bsz: {}\n".format(b))
    f.write("dev_bsz: {}\n".format(dev_bsz))
    f.write("current gpu: {}\n".format(current_gpu))
    f.write("\n")







In [71]:
if reverse_src:
    if embType == "word":

        for sent in train_src_sentences[:3]:
            print([idx_to_src_word[idx] for idx in sent])    
        train_src_sentences = [sent[::-1] for sent in train_src_sentences]   
        for sent in train_src_sentences[:3]:
            print([idx_to_src_word[idx] for idx in sent])    

        print()

        for sent in dev_src_sentences[:3]:
            print([idx_to_src_word[idx] for idx in sent])
        dev_src_sentences = [sent[::-1] for sent in dev_src_sentences]
        for sent in dev_src_sentences[:3]:
            print([idx_to_src_word[idx] for idx in sent])    

    elif embType == "jointBPE":
        for sent in train_src_sentences[:3]:
            print([idx_to_subword[idx] for idx in sent])    
        train_src_sentences = [sent[::-1] for sent in train_src_sentences]   
        for sent in train_src_sentences[:3]:
            print([idx_to_subword[idx] for idx in sent]) 

        print()

        for sent in dev_src_sentences[:3]:
            print([idx_to_subword[idx] for idx in sent])
        dev_src_sentences = [sent[::-1] for sent in dev_src_sentences]
        for sent in dev_src_sentences[:3]:
            print([idx_to_subword[idx] for idx in sent])  

trainingPairs = list(zip(train_src_sentences, train_trg_sentences))

unrolled = False
if overfit_on_train: 
    trainBatches = getBatches(trainingPairs[:10], b, device)
    devBatches = getDevBatches(train_src_sentences[:10], dev_bsz, device)
else:
    trainBatches = getBatches(trainingPairs, b, device)
    devBatches = getDevBatches(dev_src_sentences, dev_bsz, device)

sorting by trg length
encoder_inputs_tensor:
tensor([[ 24,  75,  64,  76,   6,  30,  41,  13,  77,  78,  79,  80,  81,  82,
          83,  84,  85,  36,  86,  31,  87,  88,  89,  90,  30,  36,  43,  91,
          36,  92,  93,  30,  94,  95,  96,  30,  97,  36,  98,  30,  99,  13,
          23, 100, 101,  30, 102, 103, 104,   6,   9],
        [ 31,  40,   6,  30,  41,  31,  29,  42,  43,  44,  45,  46,  47,  48,
          31,  49,  50,  51,  19,   6,   9,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 13,  23,  24,  25,  26,  27,  28,  26,  29,  30,  31,  32,  33,  34,
          35,  30,  30,  36,  13,  14,  15,  37,  38,  39,   9,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 10,  52,  30,   5,  53,   6,  30,  41,  13,   5,  19,  54,  55,  56,


In [18]:
print(trainBatches)

[((tensor([[3, 4, 5],
        [6, 7, 0],
        [2, 0, 0]], device='cuda:0'), tensor([3, 2, 1], device='cuda:0'), tensor([2, 0, 1], device='cuda:0')), (tensor([[2, 4, 5, 6],
        [2, 7, 8, 0],
        [2, 9, 0, 0]], device='cuda:0'), tensor([4, 3, 2], device='cuda:0'), tensor([[[False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False]],

        [[False, False,  True],
         [False, False,  True],
         [False, False,  True],
         [False, False,  True]],

        [[False,  True,  True],
         [False,  True,  True],
         [False,  True,  True],
         [False,  True,  True]]], device='cuda:0')), (tensor([[4, 5, 6, 3],
        [7, 8, 3, 0],
        [9, 3, 0, 0]], device='cuda:0'), tensor([4, 3, 2], device='cuda:0')))]


In [72]:

if overfit_on_train:
    translator = train(translator, optimizer, trainBatches, devBatches, train_references[:10], num_epochs=num_ep, cur_ep=0, folder=folder, save=False)
else:
    translator = train(translator, optimizer, trainBatches, devBatches, dev_references, num_epochs=num_ep, cur_ep=0, folder=folder, save=True)





/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


ep: 00, loss: 2345.11401367, ep_t: 0.05 sec, t_t: 0.28 sec, bleu: 0.0080
ep: 01, loss: 1940.06958008, ep_t: 0.03 sec, t_t: 0.28 sec, bleu: 0.0200


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


ep: 02, loss: 820.52441406, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 0.1296
ep: 03, loss: 656.14984131, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 0.0144
ep: 04, loss: 416.89974976, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 0.0722
ep: 05, loss: 231.33290100, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 0.1688
ep: 06, loss: 127.36117554, ep_t: 0.03 sec, t_t: 0.28 sec, bleu: 0.3996
ep: 07, loss: 84.70709229, ep_t: 0.03 sec, t_t: 0.28 sec, bleu: 0.2172
ep: 08, loss: 70.37828827, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 0.5717
ep: 09, loss: 40.88386154, ep_t: 0.03 sec, t_t: 0.28 sec, bleu: 0.5125
ep: 10, loss: 27.97411346, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 0.8266
ep: 11, loss: 19.75640678, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 0.8700
ep: 12, loss: 14.46535015, ep_t: 0.03 sec, t_t: 0.41 sec, bleu: 0.7200
ep: 13, loss: 10.52904892, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 0.7200
ep: 14, loss: 8.12210274, ep_t: 0.03 sec, t_t: 0.27 sec, bleu: 1.0000
ep: 15, loss: 6.10749912, ep_t: 0.03 sec, t_t: 0.28 sec, bleu: 1.0000
ep:

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

model_name = 'BPE_h500/'
folder = '/content/gdrive/My Drive/MToutputs/' + model_name

epoch = 16 # desired checkpoint to load
checkpoint = torch.load(folder + 'cp' + str(epoch) + '.tar') 
translator.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
ep_loss = checkpoint['ep_loss']
last_finished_epoch = checkpoint['epoch']
next_ep = last_finished_epoch + 1 # becomes first cur_ep value, if continue training this model
print("ep_loss: {}".format(ep_loss))
print("last_finished_epoch: {}".format(last_finished_epoch))

# perform beam search on best model:
translator.decoder.inf_alg = "beam_search"
translator.decoder.beam_size = 10

translator.eval()
test_time, drop_bleu, bleu_time = test(translator, devBatches, dev_references, folder, last_finished_epoch, write=True)
print("t_t: {}, bleu: {}".format(test_time, drop_bleu))

#!!!new experiment: overwrite the LR of the optimizer to implement a quasi-LR schedule
#for g in optimizer.param_groups:
#    g['lr'] = .00005
#optimizer.state_dict()
# continue training
#translator.train()
#translator = train(translator, optimizer, trainBatches, devBatches, dev_references, num_epochs=num_ep, cur_ep=next_ep, folder=folder, save=True)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ep_loss: 5310.45263671875
last_finished_epoch: 16
t_t: 264.22928047180176, bleu: 0.25813851467550597


In [ ]:
# compare predictions with targets

with open('/content/gdrive/My Drive/MToutputs/lstm_dr/preds10.txt', 'r') as f:
    preds = f.read()
    
    
preds_list = to_sentences([preds])
preds = preds_list[0]
for i in range(250, 300):
    print("pred:  ", preds[i])
    print("target:", ' '.join(dev_references[i][0]))
    print()

In [ ]:
!grep -n "instagram" "/content/gdrive/My Drive/MToutputs/lstm_dr/preds10.txt"

281:that's why instagram is fat .


In [ ]:
# now use parallelized version:
translator.cuda()
translator.encoder.dev = "cuda:0"
translator.decoder.dev = "cuda:0"
translator.eval()
#print("making greedy predictions on the training set...")
translator.decoder.alg = "greedy"
train_g_translations, test_time, bleu, bleu_time = test(translator, devBatches, train_references[:10])

for trans in train_g_translations:
    print(trans)

david gallo this is bill lange . i'm dave gallo .
and we're going to tell you some stories from the sea here in video .
we've got some of the most incredible video of titanic that's ever been seen , and we're not going to show you any of it .
the truth of the about is that the titanic - - even though it's breaking all sorts of box office records - - it's not the most exciting story from the sea .
and the problem , i think , is that we take the ocean for granted .
when you think about it , the oceans are 75 percent of the planet .
most of the planet is ocean water .
the average depth is about two miles .
part of the problem , i think , is we stand at the beach , or we see images like this of the ocean , and you look out at this great big blue expanse , and it's shimmering and it's moving and there's waves and there's surf and there's tides , but you have no idea for what lies in there
and in the oceans , there are the longest mountain ranges on the planet .


In [ ]:
for trans in train_g_translations:
    print(trans)

david gallo this is bill lange . i'm dave gallo .
and we're going to tell you some stories from the sea here in video .
part of the problem , i think , is we take the beach , or we see images like this of the ocean , and you look out at the great big blue expanse , and it's shimmering and it's moving and there's waves and there's surf and there's tides , but you have no idea for what lies in there .
the truth of the matter is that the titanic - - even though it's breaking all sorts of box office - - - the not not the most exciting story from the sea .
and the problem , i think , is that we take the ocean for granted .
and the problem about it , the oceans are 75 percent of the planet .
most of the planet is ocean water .
the average depth is about two miles .
part of the problem , i think , is we stand at the beach , or we see images like this of the ocean , and you look out at this great big blue expanse , and it's shimmering and it's moving and there's waves and there's surf and ther

In [ ]:
### early-stopping workaround
fold = "/content/gdrive/My Drive/models/" # folder
# for epoch 0, train the model from scratch
print("training model 0...")
translator, final_loss = train2(translator, optimizer, batches, bsz=b, num_epochs=1, cur_ep=0, learn_rate=LR, folder=fold)

# perform inference with this from-scratch model
if torch.cuda.is_available():
    translator.cpu()
    translator.encoder.dev = "cpu"
    translator.decoder.dev = "cpu"
translator.eval()
print("making greedy predictions on the dev set with model 0...")
start_time = time.time()
#dev_g_translations = test(translator, dev_src_sentences)
# only estimate the BLEU using ~1/8 of the dev set:
dev_g_translations = test(translator, dev_src_sentences[:1000])

print("...took %0.2f seconds" % (time.time()-start_time))
print()
# Create a local file to upload.
with open('/content/gdrive/My Drive/predictions/dev_greedy_preds0.txt', 'w') as f:
    for translation in dev_g_translations:
        f.write(translation + '\n')
        
        

training model 0...
epoch: 0, loss: 7208.15, time: 192.49 sec
making greedy predictions on the dev set with model 0...
...took 118.42 seconds



In [ ]:

# for epochs 1 thru 6, load previous model from file
for i in range(1,16): 
  
    # intialize models so that can load into them
    new_encoder = Encoder(srcV, input_size=IS, hidden_size=enc_hs, num_layers=NL, bsz=b, dev=device)
    new_decoder = Decoder(trgV, idx_to_trg_word, input_size=IS, hidden_size=dec_hs, num_layers=NL, attentionMechanism="global_att", realData=usingRealData, bsz=b, dev=device)
    new_translator = RNNencdec(new_encoder, new_decoder)

    if torch.cuda.is_available():
        new_translator.cuda()
    
    if opt == "RMS":
        optimizer = torch.optim.RMSprop(new_translator.parameters(), lr=LR) # initialize optimizer
    elif opt == "ADAM":
        optimizer = torch.optim.Adam(new_translator.parameters(), lr=LR) # initialize optimizer

    checkpoint = torch.load(fold + 'train_checkpoint' + str(i-1) + '.tar') # load previous checkpoint

    new_translator.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    ep_loss = checkpoint['ep_loss']
    print(ep_loss)

    new_translator.train()
    # train for an additional epoch
    print("training model " + str(i) + "...")

    new_translator, final_loss = train2(new_translator, optimizer, batches, bsz=b, num_epochs=i+1, cur_ep=i, folder=fold)

    
    # perform inference with the new model
    if torch.cuda.is_available():
        new_translator.cpu()
        new_translator.encoder.dev = "cpu"
        new_translator.decoder.dev = "cpu"
    new_translator.eval()
    print("making greedy predictions on the dev set with model " + str(i) + "...")
    start_time = time.time()
    #dev_g_translations = test(new_translator, dev_src_sentences)
    # only estimate the BLEU using ~1/8 of the dev set:
    dev_g_translations = test(new_translator, dev_src_sentences[:1000])
    print("...took %0.2f seconds" % (time.time()-start_time))
    print()
    # Create a local file to upload.
    with open('/content/gdrive/My Drive/predictions/dev_greedy_preds' + str(i) + '.txt', 'w') as f:
        for translation in dev_g_translations:
            f.write(translation + '\n')







tensor(7208.1484, device='cuda:0', requires_grad=True)
training model 1...
epoch: 1, loss: 5146.83, time: 187.54 sec
making greedy predictions on the dev set with model 1...
...took 111.95 seconds

tensor(5146.8330, device='cuda:0', requires_grad=True)
training model 2...
epoch: 2, loss: 4453.28, time: 186.93 sec
making greedy predictions on the dev set with model 2...
...took 106.26 seconds

tensor(4453.2798, device='cuda:0', requires_grad=True)
training model 3...
epoch: 3, loss: 4072.48, time: 187.05 sec
making greedy predictions on the dev set with model 3...
...took 102.36 seconds

tensor(4072.4817, device='cuda:0', requires_grad=True)
training model 4...
epoch: 4, loss: 3811.85, time: 186.74 sec
making greedy predictions on the dev set with model 4...
...took 107.40 seconds

tensor(3811.8545, device='cuda:0', requires_grad=True)
training model 5...
epoch: 5, loss: 3609.07, time: 186.73 sec
making greedy predictions on the dev set with model 5...
...took 110.42 seconds

tensor(360

KeyboardInterrupt: ignored

In [ ]:
# train a model from scratch
### NOTE - this will overwrite any existing checkpoints unless you change the path to where this train loop stores its checkpoints 
### (must change the fold param)

#srcV=20,000, trgV=15,000, dropout=.3
translator, final_loss = train2(translator, optimizer, batches, bsz=b, num_epochs=num_ep, folder=fold) # train model


epoch: 0, loss: 6262.86, time: 937.87 sec
epoch: 1, loss: 4415.96, time: 937.92 sec
epoch: 2, loss: 3752.01, time: 938.88 sec
epoch: 3, loss: 3260.38, time: 938.36 sec
epoch: 4, loss: 2819.71, time: 938.74 sec
epoch: 5, loss: 2420.68, time: 939.27 sec
epoch: 6, loss: 2073.30, time: 939.06 sec
epoch: 7, loss: 1775.67, time: 937.98 sec
epoch: 8, loss: 1537.47, time: 937.98 sec
epoch: 9, loss: 1343.11, time: 938.27 sec
epoch: 10, loss: 1189.45, time: 937.89 sec


KeyboardInterrupt: ignored

In [ ]:

# perform inference with this from-scratch model

# switch to CPU at test time:
if torch.cuda.is_available():
    translator.cpu()
    translator.encoder.dev = "cpu"
    translator.decoder.dev = "cpu"

translator.eval()
print("making greedy predictions on the dev set...")
start_time = time.time()
dev_g_translations = test(translator, dev_src_sentences)
print("...took %0.2f seconds" % (time.time()-start_time))
#g_test_accuracy = computeAccuracy(dev_g_translations, dev_trg_sentences, idx_to_trg_word)
#print("g test accuracy: " + g_test_accuracy)
print()
# Create a local file to upload.
#with open('dev_greedy_preds.txt', 'w') as f:
with open('/content/gdrive/My Drive/predictions/dev_greedy_preds.txt', 'w') as f:
    for translation in dev_g_translations:
        f.write(translation + '\n')
        print(translation)

translator.decoder.alg = "beam_search" # now switch to beam search for comparison
print("making beam search predictions on the dev set...")
start_time = time.time()
dev_b_translations = test(translator, dev_src_sentences)
print("...took %0.2f seconds" % (time.time()-start_time))
#b_test_accuracy = computeAccuracy(dev_b_translations, dev_trg_sentences, idx_to_trg_word)
#print("b test accuracy: " + b_test_accuracy)

# Create a local file to upload.
with open('/content/gdrive/My Drive/predictions/dev_beam_preds.txt', 'w') as f:
    for translation in dev_b_translations:
        f.write(translation + '\n')
        print(translation)



making greedy predictions on the dev set...
...took 1905.13 seconds


In [ ]:
# load checkpoint for inference only

# intialize models so that can load into them
new_encoder = Encoder(srcV, input_size=IS, hidden_size=enc_hs, num_layers=NL, bsz=b, dev=device)
new_decoder = Decoder(trgV, idx_to_trg_word, input_size=IS, hidden_size=dec_hs, num_layers=NL, attentionMechanism="global_att", realData=usingRealData, bsz=b, dev=device)
new_translator = RNNencdec(new_encoder, new_decoder)

if torch.cuda.is_available():
    new_translator.cuda()

if opt == "RMS":
    optimizer = torch.optim.RMSprop(new_translator.parameters(), lr=LR) # initialize optimizer
elif opt == "ADAM":
    optimizer = torch.optim.Adam(new_translator.parameters(), lr=LR) # initialize optimizer

checkpoint = torch.load(fold + 'train_checkpoint' + str(5) + '.tar') # load previous checkpoint

new_translator.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
ep_loss = checkpoint['ep_loss']
print(ep_loss)

# perform inference with the new model
if torch.cuda.is_available():
    new_translator.cpu()
    new_translator.encoder.dev = "cpu"
    new_translator.decoder.dev = "cpu"
new_translator.eval()
new_translator.decoder.alg = "beam_search" # now switch to beam search for comparison
new_translator.decoder.beam_size = 10
print("making beam search predictions on the dev set with model " + str(5) + "...")

#print("making greedy predictions on the dev set with model " + str(5) + "...")
start_time = time.time()
dev_b_translations = test(new_translator, dev_src_sentences[:1000])
print("...took %0.2f seconds" % (time.time()-start_time))
print()
# Create a local file to upload.
with open('/content/gdrive/My Drive/predictions/dev_beam_preds_bs10' + str(5) + '.txt', 'w') as f:
    for translation in dev_b_translations:
        f.write(translation + '\n')




tensor(3609.0740, device='cuda:0', requires_grad=True)
making beam search predictions on the dev set with model 5...
...took 1464.46 seconds



In [ ]:
# load checkpoint for inference only
###BPE beam search


# intialize models so that can load into them
new_encoder = Encoder(srcV, input_size=IS, hidden_size=enc_hs, num_layers=NL, bsz=b, dev=device)
new_decoder = Decoder(trgV, idx_to_trg_word, input_size=IS, hidden_size=dec_hs, num_layers=NL, attentionMechanism="global_att", realData=usingRealData, bsz=b, dev=device)
new_translator = RNNencdec(new_encoder, new_decoder)

if torch.cuda.is_available():
    new_translator.cuda()

if opt == "RMS":
    optimizer = torch.optim.RMSprop(new_translator.parameters(), lr=LR) # initialize optimizer
elif opt == "ADAM":
    optimizer = torch.optim.Adam(new_translator.parameters(), lr=LR) # initialize optimizer

checkpoint = torch.load(fold + 'train_checkpoint' + str(19) + '.tar') # load previous checkpoint

new_translator.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
ep_loss = checkpoint['ep_loss']
print(ep_loss)

# perform inference with the new model
if torch.cuda.is_available():
    new_translator.cpu()
    new_translator.encoder.dev = "cpu"
    new_translator.decoder.dev = "cpu"
new_translator.eval()
print("making greedy predictions on the test set with model " + str(19) + "...")
start_time = time.time()
dev_g_translations = test(new_translator, dev_src_sentences)
print("...took %0.2f seconds" % (time.time()-start_time))
print()
# Create a local file to upload.
with open('/content/gdrive/My Drive/predictions/dev_beam_preds_BPE' + str(19) + '.txt', 'w') as f:
    for translation in dev_g_translations:
        f.write(translation + '\n')